In [125]:
import time
import urllib.request 
import requests
from urllib import parse #UnicodeEncodeError 해결
from bs4 import BeautifulSoup 
import ssl 
import selenium
from selenium import webdriver

context = ssl._create_unverified_context() #url이 https일 때 생기는 오류를 해결해주는 코드
#search = input("검색어를 입력하세요:")
search = "아이폰12"
url = 'https://search.naver.com/search.naver?where=post&sm=tab_jum&query='
url_nextpage = "&sm=tab_pge&srchby=all&st=sim&where=post&start="
postNum = 1

url_list=[]
title_list=[]

for i in range(5): #number of pages
    url = url+parse.quote_plus(search)+url_nextpage+str(postNum)
    html = urllib.request.urlopen(url,context=context).read() 
    soup = BeautifulSoup(html,'html.parser')
    titles = soup.find_all(class_='api_txt_lines total_tit')
    for title in titles:
        title_list.append(title.text)
        url_list.append(title.attrs['href'])
    postNum+=10

In [67]:
import pandas as pd
df=pd.DataFrame({'url':url_list, 'title':title_list})

In [68]:
df.head()

,url,title
0,https://blog.naver.com/rewq2505?Redirect=Log&l...,아이폰12 PRO 가격 할인받아 개통
1,https://blog.naver.com/bushcherry1?Redirect=Lo...,아이폰12 액정필름 아이엠듀 강화유리 풀커버라 좋아요!
2,https://blog.naver.com/bubble_cook?Redirect=Lo...,아이폰 11 Pro Max 12 가격 스펙 비교 지금 써도 괜찮을까?
3,https://blog.naver.com/tech-plus?Redirect=Log&...,사진작가가 아이폰 12 Pro와 11 Pro 카메라 성능 비교해봤다?
4,https://blog.naver.com/clubkang?Redirect=Log&l...,"아이폰12 미니, 요즘 가격! 퍼플 색상은 언제?"


In [154]:
url = df['url'][1]
print(url)


https://blog.naver.com/bushcherry1?Redirect=Log&logNo=222221291137


In [160]:
res = requests.get(url)
print(f'상태코드: {res.status_code}, 헤더: {res.headers}\n, 몸통: {res.content.decode("utf-8").strip()}')

상태코드: 200, 헤더: {'Date': 'Sun, 16 May 2021 17:10:14 GMT', 'Content-Type': 'text/html;charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'close', 'Vary': 'Accept-Encoding', 'Cache-Control': 'no-cache', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'Set-Cookie': 'JSESSIONID=B8108C13A96B0D7C6ABF267C531B8E6C.jvm1; Path=/; Secure; HttpOnly', 'Content-Encoding': 'gzip', 'Server': 'nxfps', 'Referrer-policy': 'unsafe-url'}
, 몸통: <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ko">
<head>
<meta http-equiv="Pragma" content="no-cache"/>
<meta http-equiv="Expires" content="-1"/>
<meta name="robots" content="noindex,follow"/>
<meta name="referrer" content="always"/>
<meta http-equiv="content-type" content="text/html;charset=UTF-8"/>
<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
<link rel="shortcut icon" type="image/x-icon" href="/favicon.ico?3" />
<link rel="alternate" type="application/

In [157]:
html = BeautifulSoup(res.content, 'html.parser')
target_URL = html.find('iframe').get('src')
target_URL = target_URL + 'https://section.blog.naver.com/BlogHome.nhn'
print(target_URL)
target_URL = url+"?proxyReferer=https://section.blog.naver.com/BlogHome.nhn"

/PostView.nhn?blogId=bushcherry1&logNo=222221291137&from=search&redirect=Log&widgetTypeCall=true&directAccess=falsehttps://section.blog.naver.com/BlogHome.nhn


In [158]:
res = requests.get(target_URL)
html = BeautifulSoup(res.content, 'html.parser')

In [159]:
print(html)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="ko">
<head>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="-1" http-equiv="Expires"/>
<meta content="noindex,follow" name="robots"/>
<meta content="always" name="referrer"/>
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="/favicon.ico?3" rel="shortcut icon" type="image/x-icon"/>
<link href="https://rss.blog.naver.com/bushcherry1.xml" rel="alternate" title="RSS feed for bushcherry1 Blog" type="application/rss+xml"/>
<link href="https://blog.naver.com/NBlogWlwLayout.nhn?blogId=bushcherry1" rel="wlwmanifest" type="application/wlwmanifest+xml"/>
<title>유니스의 뷰티,패션,인테리어,플레이팅 공간 : 네이버 블로그</title>
</head>
<script charset="UTF-8" src="https://ssl.pstatic.net/t.static.blog/mylog/versioning/Frameset-657805386_https.js" type="text/javascript"></script>
<s

In [147]:
text = html.find('div', {'class':'se-main-container'}).get_text()
re.compile('[^가-힣0-9ㄱ-ㅎㅏ-ㅣ^]+').sub(' ', text).strip()

blog = pd.DataFrame(columns = ['head', 'user', 'date', 'text'])
blog = blog.append({'text':re.compile('\W+').sub(' ', text).strip()})

AttributeError: 'NoneType' object has no attribute 'get_text'

In [87]:
data={}

number = 3 #수집할 글의 갯수
for i in range(0, number):
    url = df['url'][i]
    driver = webdriver.Chrome(r"/Users/boralim/Desktop/capstone/chromedriver")
    driver.get(url)
    
    try:
        driver.switch_to.frame('mainFrame')
        target_info = {}
        
        #title
        overlays = ".se-fs-.se-ff-"
        tit = driver.find_element_by_css_selector(overlays)
        title = tit.text
        print(title)
        
        #date
        overlays = ".se_publishDate.pcol2"
        date = driver.find_element_by_css_selector(overlays)
        datetime = date.text
        print(datetime)
        
        #contents
        overlays = ".se-component .se-text .se-l-default"
        contents = driver.find_element_by_css_selector(overlays)
        print(ccontents)
        
        content_list=[]
        for content in contents:
            content_list.append(content.text)
        content_str = ' '.join(content_list)
        
        target_info['title'] = title
        target_info['datetime'] = datetime
        target_info['contents'] = content_str
        
        data[i] = target_info
        time.sleep(1)
        
        print(i, title)
        
        driver.close()
        
    except:
        driver.close()
        time.sleep(1)
        continue


2021. 5. 13. 18:34
아이폰12 액정필름 아이엠듀 강화유리 풀커버라 좋아요!
2021. 1. 27. 8:10
11 vs 12
2021. 5. 13. 15:46


In [88]:
data

{}